# Объекто-ориентированные шахматы +

6.	Реализовать функцию подсказки выбора новой позиции фигуры: после выбора фигуры для хода функция визуально на поле показывает поля доступные для хода или фигуры соперника, доступные для взятия, выбранной фигурой. Информация о допустимых ходах должна храниться в объектно-ориентированном виде, алгоритм без модификации должен работать при добавлении новых типов фигур (задание берется совместно с Заданием 1).
Сложность 1

7.	Реализовать функцию подсказки угрожаемых фигур: она возвращает информацию о том, какие фигуры ходящего игрока сейчас находятся под боем (т.е. могут быть взяты соперником на следующий ход) и визуально выделяет их на поле. Функция отдельно указывает на наличие шаха королю. Информация о допустимых ходах должна храниться в объектно-ориентированном виде, алгоритм без модификации должен работать при добавлении новых типов фигур (задание берется совместно с Заданием 1).
Сложность 1


In [16]:
import copy
from termcolor import colored

class Piece:
    # класс фигуры с атрибутами символ фигуры и ее цвет
    def __init__(self, color, symbol):
        self.color = color
        self.symbol = symbol

    def __str__(self):
        # чтобы возвращало не классовый объект, а сам символ
        return self.symbol

class Pawn(Piece):
    # класс пешки
    def __init__(self, color):
        # super() - это функция, которая позволяет вызывать методы 
        # родительского класса в дочернем классе
        super().__init__(color, 'P' if color == 'white' else 'p')
    # получаем доступные ходы для пешки, работая с координатами
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        if self.color == 'white':
            if board[x-1][y] == '.':
                valid_moves.append((x-1, y))
            # если пешка ходит 1ый раз, добавляем двойной ход
            if x == 6 and board[x-2][y] == '.':
                valid_moves.append((x-2, y))
            # добавляем ходы, если есть возможность срубить фигуру 
            if y > 0 and board[x-1][y-1] != '.' and board[x-1][y-1].color != self.color:
                valid_moves.append((x-1, y-1))
            if y < 7 and board[x-1][y+1] != '.' and board[x-1][y+1].color != self.color:
                valid_moves.append((x-1, y+1))
        else:
            if board[x+1][y] == '.':
                valid_moves.append((x+1, y))
            if x == 1 and board[x+2][y] == '.':
                valid_moves.append((x+2, y))
            if y > 0 and board[x+1][y-1] != '.' and board[x+1][y-1].color != self.color:
                valid_moves.append((x+1, y-1))
            if y < 7 and board[x+1][y+1] != '.' and board[x+1][y+1].color != self.color:
                valid_moves.append((x+1, y+1))
            
        return valid_moves

class Rook(Piece):
    # класс ладьи
    def __init__(self, color):
        super().__init__(color, 'R' if color == 'white' else 'r')
        # has_moved показывает, ходит ли ладья 1ый раз
        # это необходимо для рокировки
        self.has_moved=False
    # получаем возможные ходы для ладьи
    # ладья ходит по горизонталям и вертикалям во все стороны
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        # ходы вверх и вниз
        for i in range(x-1, -1, -1):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            # проверяем возможность порубить фигуру
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        for i in range(x+1, 8):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        # ходы по сторонам
        for j in range(y-1, -1, -1):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        for j in range(y+1, 8):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        return valid_moves

class Knight(Piece):
    # класс коня
    def __init__(self, color):
        super().__init__(color, 'N' if color == 'white' else 'n')
    # получаем возможные ходы для коня
    # на два поля вверх или вниз, а затем на поле вбок, 
    # либо на два поля вбок, а затем на одно поле вверх или вниз (буквой Г)
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        moves = [(x-2, y-1), (x-2, y+1), (x-1, y-2), (x-1, y+2),
                 (x+1, y-2), (x+1, y+2), (x+2, y-1), (x+2, y+1)]
        for move in moves:
            # проверяем, в пределах доски ли ход
            # пустая ли клетка там или есть возможноть срубить
            if 0 <= move[0] < 8 and 0 <= move[1] < 8 and 
            (board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                valid_moves.append(move)
        return valid_moves

class Bishop(Piece):
    # класс слона
    def __init__(self, color):
        super().__init__(color, 'B' if color == 'white' else 'b')
    # получаем возможные ходы для слона
    # слон ходит по диагонали по все стороны
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        # функция zip позволяет пройтись одновременно по нескольким итерируемым объектам (спискам и др.)
        # левые диагонали
        for i, j in zip(range(x-1, -1, -1), range(y-1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x+1, 8), range(y-1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        # правые диагонали
        for i, j in zip(range(x-1, -1, -1), range(y+1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x+1, 8), range(y+1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        return valid_moves

class Queen(Piece):
    # класс ферзя
    def __init__(self, color):
        super().__init__(color, 'Q' if color == 'white' else 'q')
    # проверяем возможные ходы для ферзя
    # ферзь ходит во всех направлениях: по диагонали и по прямой
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        # вверх и вниз
        for i in range(x-1, -1, -1):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        for i in range(x+1, 8):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        # вправо и влево
        for j in range(y-1, -1, -1):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        for j in range(y+1, 8):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        # левые диагонали
        for i, j in zip(range(x-1, -1, -1), range(y-1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x+1, 8), range(y-1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        # правые диагонали
        for i, j in zip(range(x-1, -1, -1), range(y+1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x+1, 8), range(y+1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        return valid_moves

class King(Piece):
    # класс короля
    def __init__(self, color):
        super().__init__(color, 'K' if color == 'white' else 'k')
        # has_moved определяет, ходил ли уже король
        # это необходимо для рокировки
        self.has_moved=False
    # получаем возможные ходы короля
    # король ходит на одну клетку в любую сторону
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        # стандартные ходы
        moves = [(x-1, y-1), (x-1, y), (x-1, y+1),
                 (x, y-1),           (x, y+1),
                 (x+1, y-1), (x+1, y), (x+1, y+1)]
        for move in moves:
            # проверяем, в пределах доски ли ход
            # пустая ли клетка там или есть возможноть срубить
            if 0 <= move[0] < 8 and 0 <= move[1] < 8 and 
            (board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                valid_moves.append(move)
        # проверяем на возможность рокировки
        if not self.has_moved:
            # рокировка справа
            # проверка, есть ли на нужном месте ладья, которая еще не ходила
            if board[x][7]!='.' and isinstance(board[x][7], Rook) and not board[x][7].has_moved:
                # проверка, пустые ли клетки между королем и ладьей
                if all(board[x][col]=='.' for col in range(y+1, 7)):
                    valid_moves.append((x, y+2))
                    
            # рокировка слева
            if board[x][0]!='.' and isinstance(board[x][0], Rook) and not board[x][0].has_moved:
                if all(board[x][col]=='.' for col in range(1, y)):
                    valid_moves.append((x, y-2))
        return valid_moves

class Board:
    # класс доски
    def __init__(self):
        # доска заполняется символом '.'
        self.board = [['.' for _ in range(8)] for _ in range(8)]
        self.setup_board()
    def setup_board(self):
        # расставляем белые фигуры
        self.board[7][0] = Rook('white')
        self.board[7][1] = Knight('white')
        self.board[7][2] = Bishop('white')
        self.board[7][3] = Queen('white')
        self.board[7][4] = King('white')
        self.board[7][5] = Bishop('white')
        self.board[7][6] = Knight('white')
        self.board[7][7] = Rook('white')
        for i in range(8):
            self.board[6][i] = Pawn('white')

        # расставляем черные фигуры
        self.board[0][0] = Rook('black')
        self.board[0][1] = Knight('black')
        self.board[0][2] = Bishop('black')
        self.board[0][3] = Queen('black')
        self.board[0][4] = King('black')
        self.board[0][5] = Bishop('black')
        self.board[0][6] = Knight('black')
        self.board[0][7] = Rook('black')
        for i in range(8):
            self.board[1][i] = Pawn('black')
    
    # функция для вывода доски
    def print_board(self):
        # названия столбцов
        print('  a b c d e f g h')
        # номера строк + фигуры
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                print(self.board[7-i][j], end=' ')
            print()
            
    # задание 7
    # вывод доски с выделением белых фигур под угрозой
    def print_board_white(self):
        board = [['.' for _ in range(8)] for _ in range(8)]
        for i in range(8):
            for j in range(8):
                board[i][j] = self.board[i][j]
        
        print('  a b c d e f g h')
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                piece = self.board[7-i][j]
                if piece != '.' and piece.color == 'black': # если фигура черная, получаем для нее возможные ходы
                    moves = self.board[7-i][j].get_valid_moves(self.board, (7-i,j))
                    for move in moves:
                        # если возможно срубить белую фигуру, то окрашиваем ее символ в красный
                        if board[move[0]][move[1]]!='.' and board[move[0]][move[1]]!= piece.color:
                            board[move[0]][move[1]]=colored(board[move[0]][move[1]],'red')
                print(board[7-i][j], end=' ')
            print()
        
    # вывод доски с выделением черных фигур под угрозой
    def print_board_black(self):
        board = [['.' for _ in range(8)] for _ in range(8)]
        for i in range(8):
            for j in range(8):
                board[i][j] = self.board[i][j]
             
        print('  a b c d e f g h')
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                piece = self.board[7-i][j]
                if piece!='.' and piece.color == 'white': # если фигура белая, получаем для нее возможные ходы
                    moves = piece.get_valid_moves(self.board, (7-i,j))
                    for move in moves:
                        # если возможно срубить черную фигуру, то окрашиваем ее символ в красный
                        if board[move[0]][move[1]]!='.' and board[move[0]][move[1]]!= piece.color:
                            board[move[0]][move[1]]=colored(board[move[0]][move[1]],'red')
                print(board[7-i][j], end=' ')
            print()
        
    # задание 6
    # функция вывода подсказки для хода
    def hint(self, start):
        # создаем новую доску, на которой будет подсказка
        board = [['.' for _ in range(8)] for _ in range(8)]
        dictt={'a': 0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        x, y = 8-int(start[1]), dictt[start[0]]
        piece=self.board[x][y]
        for i in range(8):
            for j in range(8):
                # если в клетку может сходить фигура и она пустая
                # она обозначается !
                if (i,j) in piece.get_valid_moves(self.board, (x,y)):
                    if self.board[i][j]=='.':
                        board[i][j]='!'
                if self.board[i][j]!='.':
                    board[i][j]=self.board[i][j]
        print('  a b c d e f g h')
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                print(board[7-i][j], end=' ')
            print()
                        
    # функция движения фигуры
    def move_piece(self, start, end):
        # словарь для распознавания букв
        dictt = {'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        # проверка, возможен ли ход
        if self.is_valid_move(start, end):
            # если ходит ладья или король, меняем значение has_moved на True
            if isinstance(self.board[8-int(start[1])][dictt[start[0]]], Rook):
                Rook.has_moved = True
            if isinstance(self.board[8-int(start[1])][dictt[start[0]]], King):
                King.has_moved = True
                # проверяем, делаем ли мы рокировку справа
                if end=='g1' or end=='g8':
                    # двигаем ладью
                    self.board[8-int(start[1])][dictt['f']] = self.board[8-int(start[1])][dictt['h']]
                    self.board[8-int(start[1])][dictt['h']] = '.'

                # проверяем, делаем ли мы рокировку слева
                elif end=='c1' or end=='c8':
                    # двигаем ладью
                    self.board[8-int(start[1])][dictt['d']] = self.board[8-int(start[1])][dictt['a']]
                    self.board[8-int(start[1])][dictt['a']] = '.'

            # двигаем фигуру
            self.board[8-int(end[1])][dictt[end[0]]] = self.board[8-int(start[1])][dictt[start[0]]]
            self.board[8-int(start[1])][dictt[start[0]]] = '.'
            return True
        else:
            return False
                                                                               
    # функция, для проверки позможности хода
    def is_valid_move(self, start, end):
        # словарь для распознавания букв
        dictt={'a': 0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        x1, y1 = start
        x2, y2 = end
        # меняем значения на нужные
        # у отвечает за строку, х - за столбец
        x1,x2=dictt[x1], dictt[x2]
        y1,y2=8-int(y1),8-int(y2)
        start=y1,x1
        # проверка, в пределах доски ли ход
        if 0 <= x1 < 8 and 0 <= y1 < 8 and 0 <= x2 < 8 and 0 <= y2 < 8:
            piece = self.board[y1][x1]
            # проверка, есть ли фигура на стартовой позиции
            if piece != '.':
                # получаем возможные ходы для фигуры
                valid_moves = piece.get_valid_moves(self.board, start)
                # возращаем True, если конечная позиция в возможных ходах
                # False, если нельзя сходить в конечную позицию
                return (y2, x2) in valid_moves
        return False
    
    # функция для проверки шаха
    def is_check(self, color):
        # получаем координаты короля
        king_position = self.get_king_position(color)
        for i in range(8):
            for j in range(8):
                # проходимся по доске
                piece = self.board[i][j]
                # если находим фигуру другого цвета
                if piece != '.' and piece.color != color:
                    # проверяем есть ли координаты короля в возможных ходах для фигуры
                    valid_moves = piece.get_valid_moves(self.board, (i, j))
                    if king_position in valid_moves:
                        return True
        return False
    
    # функция для проверки мата
    def is_checkmate(self, color):
        # выполняется, только если король уже под шахом
        if not self.is_check(color):
            return False

        # проверяем возможные ходы для короля, чтобы избежать шаха
        king_position = self.get_king_position(color)
        valid_moves = self.board[king_position[0]][king_position[1]].get_valid_moves(self.board, king_position)
        piece = self.board[king_position[0]][king_position[1]]
        for move in valid_moves:
            # делаем временный ход, и проверем попадает ли он под шах
            self.board[move[0]][move[1]] = piece
            self.board[king_position[0]][king_position[1]] = '.'
            if not self.is_check(color):
                # возвращаем фигуры назад
                self.board[king_position[0]][king_position[1]] = piece
                self.board[move[0]][move[1]] = '.'
                return False
            # возвращаем фигуры назад
            self.board[king_position[0]][king_position[1]] = piece
            self.board[move[0]][move[1]] = '.'


        # проверяем возможные ходы для других фигур, которые спасаю короля от шаха
        for i in range(8):
            for j in range(8):
                piece = self.board[i][j]
                if piece != '.' and piece.color == color:
                    for move in piece.get_valid_moves(self.board, (i, j)):
                        # делаем временный ход
                        self.board[move[0]][move[1]] = piece
                        self.board[i][j] = '.'
                        if not self.is_check(color):
                            # возвращаем фигуры назад
                            self.board[i][j] = piece
                            self.board[move[0]][move[1]] = '.'
                            return False
                        # возвращаем фигуры назад
                        self.board[i][j] = piece
                        self.board[move[0]][move[1]] = '.'
        return True

    # функция для получения координатов короля
    def get_king_position(self, color):
        for i in range(8):
            for j in range(8):
                piece = self.board[i][j]
                if piece != '.' and piece.color == color and isinstance(piece, King):
                    return (i, j)
        return None

# функция для игры    
def play_game():
    # создаем доску
    board = Board()
    board.print_board()

    while True:
        # получаем ход белых
        while True:
            start = input("Ход белых (например, e2): ")
            print('Подсказка для хода')
            board.hint(start)
            end = input("Ход белых (например, e4): ")
            if board.is_valid_move(start, end):
                break
            else:
                print("Неправильный ход. Попробуйте снова.")

        # ходим белыми
        board.move_piece(start, end)
        board.print_board_black()

        # проверяем, находятся ли черные под матом
        if board.is_checkmate('black'):
            print("Белые выиграли")
            break

        # проверяем, находятся ли черные под шахом
        if board.is_check('black'):
            print("У черных шах")

        # получаем ход черных
        while True:
            start = input("Ход черных (например, e7): ")
            print('Подсказка для хода')
            board.hint(start)
            end = input("Ход черных (например, e5): ")
            if board.is_valid_move(start, end):
                break
            else:
                print("Неправильный ход. Попробуйте снова.")

        # ходим черными
        board.move_piece(start, end)
        board.print_board_white()

        # проверяем, находятся ли белые под матом
        if board.is_checkmate('white'):
            print("Черные выиграли!")
            break

        # проверяем, находятся ли белые под шахом
        if board.is_check('white'):
            print("У белых шах!")


play_game()


  a b c d e f g h
8 r n b q k b n r 
7 p p p p p p p p 
6 . . . . . . . . 
5 . . . . . . . . 
4 . . . . . . . . 
3 . . . . . . . . 
2 P P P P P P P P 
1 R N B Q K B N R 
Ход белых (например, e2): f2
Подсказка для хода
  a b c d e f g h
8 r n b q k b n r 
7 p p p p p p p p 
6 . . . . . . . . 
5 . . . . . . . . 
4 . . . . . ! . . 
3 . . . . . ! . . 
2 P P P P P P P P 
1 R N B Q K B N R 
Ход белых (например, e4): f4
  a b c d e f g h
8 r n b q k b n r 
7 p p p p p p p p 
6 . . . . . . . . 
5 . . . . . . . . 
4 . . . . . P . . 
3 . . . . . . . . 
2 P P P P P . P P 
1 R N B Q K B N R 
Ход черных (например, e7): e7
Подсказка для хода
  a b c d e f g h
8 r n b q k b n r 
7 p p p p p p p p 
6 . . . . ! . . . 
5 . . . . ! . . . 
4 . . . . . P . . 
3 . . . . . . . . 
2 P P P P P . P P 
1 R N B Q K B N R 
Ход черных (например, e5): e6
  a b c d e f g h
8 r n b q k b n r 
7 p p p p . p p p 
6 . . . . p . . . 
5 . . . . . . . . 
4 . . . . . P . . 
3 . . . . . . . . 
2 P P P P P . P P 
1 R N B Q K B

1.	Придумать 3 новых вида фигур с оригинальными правилами перемещения и реализовать их классы. Создать модификацию шахмат с новыми фигурами с минимальным вмешательством в существующий код.
Сложность 1

In [20]:
import copy

class Piece:
    # класс фигуры с атрибутами символ фигуры и ее цвет
    def __init__(self, color, symbol):
        self.color = color
        self.symbol = symbol

    def __str__(self):
        # чтобы возвращало не классовый объект, а сам символ
        return self.symbol

class Pawn(Piece):
    # класс пешки
    def __init__(self, color):
        # super() - это функция, которая позволяет вызывать методы 
        # родительского класса в дочернем классе
        super().__init__(color, 'P' if color == 'white' else 'p')
    # получаем доступные ходы для пешки, работая с координатами
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        if self.color == 'white':
            if board[x-1][y] == '.':
                valid_moves.append((x-1, y))
            # если пешка ходит 1ый раз, добавляем двойной ход
            if x == 6 and board[x-2][y] == '.':
                valid_moves.append((x-2, y))
            # добавляем ходы, если есть возможность срубить фигуру 
            if y > 0 and board[x-1][y-1] != '.' and board[x-1][y-1].color != self.color:
                valid_moves.append((x-1, y-1))
            if y < 7 and board[x-1][y+1] != '.' and board[x-1][y+1].color != self.color:
                valid_moves.append((x-1, y+1))
        else:
            if board[x+1][y] == '.':
                valid_moves.append((x+1, y))
            if x == 1 and board[x+2][y] == '.':
                valid_moves.append((x+2, y))
            if y > 0 and board[x+1][y-1] != '.' and board[x+1][y-1].color != self.color:
                valid_moves.append((x+1, y-1))
            if y < 7 and board[x+1][y+1] != '.' and board[x+1][y+1].color != self.color:
                valid_moves.append((x+1, y+1))
            
        return valid_moves

# 3 новые фигуры
class Spider(Piece):
    # ! ! . ! !
    # . . s . .
    # ! ! . ! !
    def __init__(self, color):
        super().__init__(color, 'S' if color == 'white' else 's')
        
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        moves=[(x-1,y-1),(x-1,y-2),(x-1,y+1),(x-1,y+2),
               (x+1,y-1),(x+1,y-2),(x+1,y+1),(x+1,y+2)]
        for move in moves:
            # проверяем, в пределах доски ли ход
            # пустая ли клетка там или есть возможноть срубить
            if 0 <= move[0] < 8 and 0 <= move[1] < 8 and 
            (board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                valid_moves.append(move)
        return valid_moves
            
class Helicopter(Piece):
    # . ! . ! .
    # ! . h . !
    # . ! . ! .
    def __init__(self, color):
        super().__init__(color, 'H' if color == 'white' else 'h')
        
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        moves = [(x-1,y-1),(x,y-2),(x+1,y-1),
                 (x-1,y+1),(x,y+2),(x+1,y+1)]
        for move in moves:
            # проверяем, в пределах доски ли ход
            # пустая ли клетка там или есть возможноть срубить
            if 0 <= move[0] < 8 and 0 <= move[1] < 8 and 
            (board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                valid_moves.append(move)
        return valid_moves

class Love(Piece):
    # ! ! . ! !
    # . ! L ! .
    # . . ! . .
    def __init__(self, color):
        super().__init__(color, 'L' if color == 'white' else 'l')
    
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        if self.color == 'white':
            moves = [(x+1,y),(x,y-1),(x,y+1),(x-1,y-1),
                     (x-1,y-2),(x-1,y+1),(x-1,y+2)]
            for move in moves:
                # проверяем, в пределах доски ли ход
                # пустая ли клетка там или есть возможноть срубить
                if 0 <= move[0] < 8 and 0 <= move[1] < 8 and 
                (board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                    valid_moves.append(move)
        if self.color == 'black':
            moves = [(x-1,y),(x,y+1),(x,y-1),(x+1,y-1),
                     (x+1,y-2),(x+1,y+1),(x+1,y+2)] 
            for move in moves:
                # проверяем, в пределах доски ли ход
                # пустая ли клетка там или есть возможноть срубить
                if 0 <= move[0] < 8 and 0 <= move[1] < 8 and 
                (board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                    valid_moves.append(move)
        return valid_moves
    
class Rook(Piece):
    # класс ладьи
    def __init__(self, color):
        super().__init__(color, 'R' if color == 'white' else 'r')
        # has_moved показывает, ходит ли ладья 1ый раз
        # это необходимо для рокировки
        self.has_moved=False
    # получаем возможные ходы для ладьи
    # ладья ходит по горизонталям и вертикалям во все стороны
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        # ходы вверх и вниз
        for i in range(x-1, -1, -1):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            # проверяем возможность порубить фигуру
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        for i in range(x+1, 8):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        # ходы по сторонам
        for j in range(y-1, -1, -1):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        for j in range(y+1, 8):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        return valid_moves

class Knight(Piece):
    # класс коня
    def __init__(self, color):
        super().__init__(color, 'N' if color == 'white' else 'n')
    # получаем возможные ходы для коня
    # на два поля вверх или вниз, а затем на поле вбок, 
    # либо на два поля вбок, а затем на одно поле вверх или вниз
    # (буквой Г)
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        moves = [(x-2, y-1), (x-2, y+1), (x-1, y-2), (x-1, y+2),
                 (x+1, y-2), (x+1, y+2), (x+2, y-1), (x+2, y+1)]
        for move in moves:
            # проверяем, в пределах доски ли ход
            # пустая ли клетка там или есть возможноть срубить
            if 0 <= move[0] < 8 and 0 <= move[1] < 8 and 
            (board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                valid_moves.append(move)
        return valid_moves

class Bishop(Piece):
    # класс слона
    def __init__(self, color):
        super().__init__(color, 'B' if color == 'white' else 'b')
    # получаем возможные ходы для слона
    # слон ходит по диагонали по все стороны
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        # функция zip позволяет пройтись одновременно по нескольким итерируемым объектам (спискам и др.)
        # левые диагонали
        for i, j in zip(range(x-1, -1, -1), range(y-1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x+1, 8), range(y-1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        # правые диагонали
        for i, j in zip(range(x-1, -1, -1), range(y+1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x+1, 8), range(y+1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        return valid_moves

class Queen(Piece):
    # класс ферзя
    def __init__(self, color):
        super().__init__(color, 'Q' if color == 'white' else 'q')
    # проверяем возможные ходы для ферзя
    # ферзь ходит во всех направлениях: по диагонали и по прямой
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        # вверх и вниз
        for i in range(x-1, -1, -1):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        for i in range(x+1, 8):
            if board[i][y] == '.':
                valid_moves.append((i, y))
            elif board[i][y].color != self.color:
                valid_moves.append((i, y))
                break
            else:
                break
        # вправо и влево
        for j in range(y-1, -1, -1):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        for j in range(y+1, 8):
            if board[x][j] == '.':
                valid_moves.append((x, j))
            elif board[x][j].color != self.color:
                valid_moves.append((x, j))
                break
            else:
                break
        # левые диагонали
        for i, j in zip(range(x-1, -1, -1), range(y-1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x+1, 8), range(y-1, -1, -1)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        # правые диагонали
        for i, j in zip(range(x-1, -1, -1), range(y+1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        for i, j in zip(range(x+1, 8), range(y+1, 8)):
            if board[i][j] == '.':
                valid_moves.append((i, j))
            elif board[i][j].color != self.color:
                valid_moves.append((i, j))
                break
            else:
                break
        return valid_moves

class King(Piece):
    # класс короля
    def __init__(self, color):
        super().__init__(color, 'K' if color == 'white' else 'k')
        # has_moved определяет, ходил ли уже король
        # это необходимо для рокировки
        self.has_moved=False
    # получаем возможные ходы короля
    # король ходит на одну клетку в любую сторону
    def get_valid_moves(self, board, position):
        valid_moves = []
        x, y = position
        # стандартные ходы
        moves = [(x-1, y-1), (x-1, y), (x-1, y+1),
                 (x, y-1),           (x, y+1),
                 (x+1, y-1), (x+1, y), (x+1, y+1)]
        for move in moves:
            # проверяем, в пределах доски ли ход
            # пустая ли клетка там или есть возможноть срубить
            if 0 <= move[0] < 8 and 0 <= move[1] < 8 and 
            (board[move[0]][move[1]] == '.' or board[move[0]][move[1]].color != self.color):
                valid_moves.append(move)
        # проверяем на возможность рокировки
        if not self.has_moved:
            # рокировка справа
            # проверка, есть ли на нужном месте ладья, которая еще не ходила
            if board[x][7]!='.' and isinstance(board[x][7], Rook) and not board[x][7].has_moved:
                # проверка, пустые ли клетки между королем и ладьей
                if all(board[x][col]=='.' for col in range(y+1, 7)):
                    valid_moves.append((x, y+2))
                    
            # рокировка слева
            if board[x][0]!='.' and isinstance(board[x][0], Rook) and not board[x][0].has_moved:
                if all(board[x][col]=='.' for col in range(1, y)):
                    valid_moves.append((x, y-2))
        return valid_moves

class Board:
    # класс доски
    def __init__(self):
        # доска заполняется символом '.'
        self.board = [['.' for _ in range(8)] for _ in range(8)]
        self.setup_board()
    def setup_board(self):
        # расставляем белые фигуры
        self.board[7][0] = Rook('white')
        self.board[7][1] = Knight('white')
        self.board[7][2] = Bishop('white')
        self.board[7][3] = Queen('white')
        self.board[7][4] = King('white')
        self.board[7][5] = Bishop('white')
        self.board[7][6] = Helicopter('white')
        self.board[7][7] = Rook('white')
        for i in range(8):
            if i==3:
                self.board[6][i] = Love('white')
            elif i==4:
                self.board[6][i] = Spider('white')
            else:
                self.board[6][i] = Pawn('white')

        # расставляем черные фигуры
        self.board[0][0] = Rook('black')
        self.board[0][1] = Knight('black')
        self.board[0][2] = Bishop('black')
        self.board[0][3] = Queen('black')
        self.board[0][4] = King('black')
        self.board[0][5] = Bishop('black')
        self.board[0][6] = Helicopter('black')
        self.board[0][7] = Rook('black')
        for i in range(8):
            if i==3:
                self.board[1][i] = Love('black')
            elif i==4:
                self.board[1][i] = Spider('black')
            else:
                self.board[1][i] = Pawn('black')
    
    # функция для вывода доски
    def print_board(self):
        # названия столбцов
        print('  a b c d e f g h')
        # номера строк + фигуры
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                print(self.board[7-i][j], end=' ')
            print()
            
    # вывод доски с выделением белых фигур под угрозой
    def print_board_white(self):
        board = [['.' for _ in range(8)] for _ in range(8)]
        for i in range(8):
            for j in range(8):
                board[i][j] = self.board[i][j]
        
        print('  a b c d e f g h')
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                piece = self.board[7-i][j]
                if piece != '.' and piece.color == 'black': # если фигура черная, получаем для нее возможные ходы
                    moves = self.board[7-i][j].get_valid_moves(self.board, (7-i,j))
                    for move in moves:
                        # если возможно срубить белую фигуру, то окрашиваем ее символ в красный
                        if board[move[0]][move[1]]!='.' and board[move[0]][move[1]]!= piece.color:
                            board[move[0]][move[1]]=colored(board[move[0]][move[1]],'red')
                print(board[7-i][j], end=' ')
            print()
        
    # вывод доски с выделением черных фигур под угрозой
    def print_board_black(self):
        board = [['.' for _ in range(8)] for _ in range(8)]
        for i in range(8):
            for j in range(8):
                board[i][j] = self.board[i][j]
             
        print('  a b c d e f g h')
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                piece = self.board[7-i][j]
                if piece!='.' and piece.color == 'white': # если фигура белая, получаем для нее возможные ходы
                    moves = piece.get_valid_moves(self.board, (7-i,j))
                    for move in moves:
                        # если возможно срубить черную фигуру, то окрашиваем ее символ в красный
                        if board[move[0]][move[1]]!='.' and board[move[0]][move[1]]!= piece.color:
                            board[move[0]][move[1]]=colored(board[move[0]][move[1]],'red')
                print(board[7-i][j], end=' ')
            print()
            
    def hint(self, start):
        board = [['.' for _ in range(8)] for _ in range(8)]
        dictt={'a': 0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        x, y = 8-int(start[1]), dictt[start[0]]
        piece=self.board[x][y]
        for i in range(8):
            for j in range(8):
                if (i,j) in piece.get_valid_moves(self.board, (x,y)):
                    if self.board[i][j]=='.':
                        board[i][j]='!'
                if self.board[i][j]!='.':
                    board[i][j]=self.board[i][j]
        print('  a b c d e f g h')
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                print(board[7-i][j], end=' ')
            print()
            
    # функция движения фигуры
    def move_piece(self, start, end):
        # словарь для распознавания букв
        dictt = {'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        x1, y1 = 8-int(start[1]), dictt[start[0]]
        x2, y2 = 8-int(end[1]), dictt[end[0]]
        # проверка, возможен ли ход
        if self.is_valid_move(start, end):
            # если ходит ладья или король, меняем значение has_moved на True
            if isinstance(self.board[x1][y1], Rook):
                Rook.has_moved = True
            if isinstance(self.board[x1][y1], King):
                King.has_moved = True
                # проверяем, делаем ли мы рокировку справа
                if end=='g1' or end=='g8':
                    # двигаем ладью
                    self.board[x1][dictt['f']] = self.board[x1][dictt['h']]
                    self.board[x1][dictt['h']] = '.'

                # проверяем, делаем ли мы рокировку слева
                elif end=='c1' or end=='c8':
                    # двигаем ладью
                    self.board[x1][dictt['d']] = self.board[x1][dictt['a']]
                    self.board[x1][dictt['a']] = '.'

            # двигаем фигуру
            self.board[x2][y2] = self.board[x1][y1]
            self.board[x1][y1] = '.'
            return True
        else:
            return False
                                                                               
    # функция, для проверки позможности хода
    def is_valid_move(self, start, end):
        # словарь для распознавания букв
        dictt={'a': 0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        x1, y1 = start
        x2, y2 = end
        # меняем значения на нужные
        # у отвечает за строку, х - за столбец
        x1,x2=dictt[x1], dictt[x2]
        y1,y2=8-int(y1),8-int(y2)
        start=y1,x1
        # проверка, в пределах доски ли ход
        if 0 <= x1 < 8 and 0 <= y1 < 8 and 0 <= x2 < 8 and 0 <= y2 < 8:
            piece = self.board[y1][x1]
            # проверка, есть ли фигура на стартовой позиции
            if piece != '.':
                # получаем возможные ходы для фигуры
                valid_moves = piece.get_valid_moves(self.board, start)
                # возращаем True, если конечная позиция в возможных ходах
                # False, если нельзя сходить в конечную позицию
                return (y2, x2) in valid_moves
        return False
    
    # функция для проверки шаха
    def is_check(self, color):
        # получаем координаты короля
        king_position = self.get_king_position(color)
        for i in range(8):
            for j in range(8):
                # проходимся по доске
                piece = self.board[i][j]
                # если находим фигуру другого цвета
                if piece != '.' and piece.color != color:
                    # проверяем есть ли координаты короля в возможных ходах для фигуры
                    valid_moves = piece.get_valid_moves(self.board, (i, j))
                    if king_position in valid_moves:
                        return True
        return False
    
    # функция для проверки мата
    def is_checkmate(self, color):
        # выполняется, только если король уже под шахом
        if not self.is_check(color):
            return False

        # проверяем возможные ходы для короля, чтобы избежать шаха
        king_position = self.get_king_position(color)
        valid_moves = self.board[king_position[0]][king_position[1]].get_valid_moves(self.board, king_position)
        piece = self.board[king_position[0]][king_position[1]]
        for move in valid_moves:
            # делаем временный ход, и проверем попадает ли он под шах
            self.board[move[0]][move[1]] = piece
            self.board[king_position[0]][king_position[1]] = '.'
            if not self.is_check(color):
                # возвращаем фигуры назад
                self.board[king_position[0]][king_position[1]] = piece
                self.board[move[0]][move[1]] = '.'
                return False
            # возвращаем фигуры назад
            self.board[king_position[0]][king_position[1]] = piece
            self.board[move[0]][move[1]] = '.'


        # проверяем возможные ходы для других фигур, которые спасаю короля от шаха
        for i in range(8):
            for j in range(8):
                piece = self.board[i][j]
                if piece != '.' and piece.color == color:
                    for move in piece.get_valid_moves(self.board, (i, j)):
                        # делаем временный ход
                        self.board[move[0]][move[1]] = piece
                        self.board[i][j] = '.'
                        if not self.is_check(color):
                            # возвращаем фигуры назад
                            self.board[i][j] = piece
                            self.board[move[0]][move[1]] = '.'
                            return False
                        # возвращаем фигуры назад
                        self.board[i][j] = piece
                        self.board[move[0]][move[1]] = '.'
        return True

    # функция для получения координатов короля
    def get_king_position(self, color):
        for i in range(8):
            for j in range(8):
                piece = self.board[i][j]
                if piece != '.' and piece.color == color and isinstance(piece, King):
                    return (i, j)
        return None

# функция для игры    
def play_game():
    # создаем доску
    board = Board()
    board.print_board()

    while True:
        # получаем ход белых
        while True:
            start = input("Ход белых (например, e2): ")
            print('Подсказка для хода')
            board.hint(start)
            end = input("Ход белых (например, e4): ")
            if board.is_valid_move(start, end):
                break
            else:
                print("Неправильный ход. Попробуйте снова.")

        # ходим белыми
        board.move_piece(start, end)
        board.print_board_black()

        # проверяем, находятся ли черные под матом
        if board.is_checkmate('black'):
            print("Белые выиграли")
            break

        # проверяем, находятся ли черные под шахом
        if board.is_check('black'):
            print("У черных шах")

        # получаем ход черных
        while True:
            start = input("Ход черных (например, e7): ")
            print('Подсказка для хода')
            board.hint(start)
            end = input("Ход черных (например, e5): ")
            if board.is_valid_move(start, end):
                break
            else:
                print("Неправильный ход. Попробуйте снова.")

        # ходим черными
        board.move_piece(start, end)
        board.print_board_white()

        # проверяем, находятся ли белые под матом
        if board.is_checkmate('white'):
            print("Черные выиграли!")
            break

        # проверяем, находятся ли белые под шахом
        if board.is_check('white'):
            print("У белых шах!")


play_game()

  a b c d e f g h
8 r n b q k b h r 
7 p p p l s p p p 
6 . . . . . . . . 
5 . . . . . . . . 
4 . . . . . . . . 
3 . . . . . . . . 
2 P P P L S P P P 
1 R N B Q K B H R 
Ход белых (например, e2): b2
Подсказка для хода
  a b c d e f g h
8 r n b q k b h r 
7 p p p l s p p p 
6 . . . . . . . . 
5 . . . . . . . . 
4 . ! . . . . . . 
3 . ! . . . . . . 
2 P P P L S P P P 
1 R N B Q K B H R 
Ход белых (например, e4): b4
  a b c d e f g h
8 r n b q k b h r 
7 p p p l s p p p 
6 . . . . . . . . 
5 . . . . . . . . 
4 . P . . . . . . 
3 . . . . . . . . 
2 P . P L S P P P 
1 R N B Q K B H R 
Ход черных (например, e7): d7
Подсказка для хода
  a b c d e f g h
8 r n b q k b h r 
7 p p p l s p p p 
6 . ! ! . ! ! . . 
5 . . . . . . . . 
4 . P . . . . . . 
3 . . . . . . . . 
2 P . P L S P P P 
1 R N B Q K B H R 
Ход черных (например, e5): c6
  a b c d e f g h
8 r n b q k b h r 
7 p p p . s p p p 
6 . . l . . . . . 
5 . . . . . . . . 
4 . P . . . . . . 
3 . . . . . . . . 
2 P . P L S P P P 
1 R N B Q K B

KeyboardInterrupt: Interrupted by user

2.	На базе игры в шахматы реализовать игру в шашки. Разработать модификацию шахмат с минимальным вмешательством в существующий код.
Сложность 2


In [ ]:
import copy

class Piece:
    # класс фигуры с атрибутами символ фигуры и ее цвет
    def __init__(self, color, symbol):
        self.color = color
        self.symbol = symbol

    def __str__(self):
        # чтобы возвращало не классовый объект, а сам символ
        return self.symbol

class Checker(Piece):
    def __init__(self, color):
        super().__init__(color, 'C' if color == 'white' else 'c')
    
    def get_valid_moves(self, board, position):
        valid_moves = []
        x,y = position
        if self.color == 'white':
            # обычные ходы по диагонали на один
            if y>0 and board[x-1][y-1] == '.':
                valid_moves.append((x-1,y-1))
            if y<7 and board[x-1][y+1] == '.':
                valid_moves.append((x-1,y+1))
            # проверяем возможность срубить
            if y>1 and board[x-1][y-1]!= '.' and board[x-1][y-1].color != self.color and board[x-2][y-2] == '.':
                valid_moves.append((x-2,y-2))
            if y<6 and board[x-1][y+1]!= '.' and board[x-1][y+1].color != self.color and board[x-2][y+2] == '.':
                valid_moves.append((x-2,y+2))
            if y>1 and board[x+1][y-1]!='.' and board[x+1][y-1].color != self.color and board[x+2][y-2] == '.':
                valid_moves.append((x+2,y-2))
            if y<6 and board[x+1][y+1]!='.' and board[x+1][y+1].color != self.color and board[x+2][y+2] == '.':
                valid_moves.append((x+2,y+2))
        else:
             # обычные ходы по диагонали на один
            if y<7 and board[x+1][y+1] == '.':
                valid_moves.append((x+1,y+1))
            if y>0 and board[x+1][y-1] == '.':
                valid_moves.append((x+1,y-1))
            # проверяем возможность срубить
            if y>1 and board[x-1][y-1]!= '.' and board[x-1][y-1].color != self.color and board[x-2][y-2] == '.':
                valid_moves.append((x-2,y-2))
            if y<6 and board[x-1][y+1]!= '.' and board[x-1][y+1].color != self.color and board[x-2][y+2] == '.':
                valid_moves.append((x-2,y+2))
            if y>1 and board[x+1][y-1]!='.' and board[x+1][y-1].color != self.color and board[x+2][y-2] == '.':
                valid_moves.append((x+2,y-2))
            if y<6 and board[x+1][y+1]!='.' and board[x+1][y+1].color != self.color and board[x+2][y+2] == '.':
                valid_moves.append((x+2,y+2))
        
        return valid_moves
        


class Board:
    # класс доски
    def __init__(self):
        # доска заполняется символом '.'
        self.board = [['.' for _ in range(8)] for _ in range(8)]
        self.setup_board()
    def setup_board(self):
        # расставляем белые шашки
        for j in range(5,8):
            if j%2==0:
                for i in range(1,8,2):
                    self.board[j][i] = Checker('white')
            else:
                for i in range(0,8,2):
                    self.board[j][i] = Checker('white')
            
        # расставляем черные шашки
        for j in range(3):
            if j%2==0:
                for i in range(1,8,2):
                    self.board[j][i] = Checker('black')
            else:
                for i in range(0,8,2):
                    self.board[j][i] = Checker('black')
    
    # функция для вывода доски
    def print_board(self):
        # названия столбцов
        print('  a b c d e f g h')
        # номера строк + фигуры
        for i in range(7, -1,-1):
            print(f'{i+1} ', end='')
            for j in range(8):
                print(self.board[7-i][j], end=' ')
            print()
            
    # функция движения фигуры
    def move_piece(self, start, end):
        # словарь для распознавания букв
        dictt = {'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        x1, y1 = 8-int(start[1]), dictt[start[0]]
        x2, y2 = 8-int(end[1]), dictt[end[0]]
        # проверка, возможен ли ход
        if self.is_valid_move(start, end):
            # проверяем, если срубили и удаляем срубленную шашку
            if abs(x2-x1)==2 and abs(y2-y1)==2:
                if (y2-y1)==-2:
                    if (x2-x1)==-2:
                        self.board[x1-1][y1-1] = '.'
                    else:
                        self.board[x1+1][y1-1] = '.'
                else:
                    if (x2-x1)==-2:
                        self.board[x1-1][y1+1] = '.'
                    else:
                        self.board[x1+1][y1+1] = '.'
    
            # двигаем фигуру
            self.board[x2][y2] = self.board[x1][y1]
            self.board[x1][y1] = '.'
            return True
        else:
            return False
                                                                               
    # функция, для проверки позможности хода
    def is_valid_move(self, start, end):
        # словарь для распознавания букв
        dictt={'a': 0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        x1, y1 = start
        x2, y2 = end
        # меняем значения на нужные
        # у отвечает за строку, х - за столбец
        x1,x2=dictt[x1], dictt[x2]
        y1,y2=8-int(y1),8-int(y2)
        start=y1,x1
        # проверка, в пределах доски ли ход
        if 0 <= x1 < 8 and 0 <= y1 < 8 and 0 <= x2 < 8 and 0 <= y2 < 8:
            piece = self.board[y1][x1]
            # проверка, есть ли фигура на стартовой позиции
            if piece != '.':
                # получаем возможные ходы для фигуры
                valid_moves = piece.get_valid_moves(self.board, start)
                # возращаем True, если конечная позиция в возможных ходах
                # False, если нельзя сходить в конечную позицию
                return (y2, x2) in valid_moves
        return False
    
    # функция для проверки, можем ли мы срубить еще одну шашку
    def go_again(self, start, end):
        dictt={'a': 0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7}
        x1, y1 = 8-int(start[1]), dictt[start[0]]
        x2, y2 = 8-int(end[1]), dictt[end[0]]
        # если до этого мы срубили шашку (т.е. разница в координатах = 2)
        if abs(x2-x1)==2 and abs(y2-y1)==2:
            # проверка есть ли такая шашка вокруг, которую можно срубить (она другого цвета и после нее пустое место)
            if (self.board[x2-1][y2-1]!='.' and self.board[x2-1][y2-1].color != self.board[x2][y2].color) or (self.board[x2-1][y2+1]!='.' and self.board[x2-1][y2+1].color != self.board[x2][y2].color):
                if self.board[x2-2][y2-2]!='.' or self.board[x2-2][y2+2]!='.':
                    return False
                return True
            elif (self.board[x2+1][y2-1]!='.' and self.board[x2+1][y2-1].color != self.board[x2][y2].color) or (self.board[x2+1][y2+1]!='.' and self.board[x2+1][y2+1].color != self.board[x2][y2].color):
                if self.board[x2+2][y2+2]!='.' or self.board[x2+2][y2-2]!='.':
                    return False
                return True
        return False
    
    # функция для проверки, закончилась ли игра
    def check_winner(self):
        # считаем количество шашек каждого цвета
        white_count = 0
        black_count = 0
        for row in self.board:
            for piece in row:
                if isinstance(piece, Checker):
                    if piece.color == 'white':
                        white_count += 1
                    else:
                        black_count += 1
        if white_count == 0 or black_count == 0:
            return True
        else:
            return False
    

# функция для игры    
def play_game():
    # создаем доску
    board = Board()
    board.print_board()

    while True:
        # получаем ход белых
        while True:
            start = input("Ход белых: ")
            end = input("Ход белых: ")
            if board.is_valid_move(start, end):
                break
            else:
                print("Неправильный ход. Попробуйте снова.")

        # ходим белыми
        board.move_piece(start, end)
        board.print_board()
        
        # проверяем возможность срубить несколько шашек
        while board.go_again(start, end):
            while True:
                start = input("Ход белых: ")
                end = input("Ход белых: ")
                if board.is_valid_move(start, end):
                    break
                else:
                    print("Неправильный ход. Попробуйте снова.")
            # ходим белыми
            board.move_piece(start, end)
            board.print_board()
            
        # проверяем, выиграли ли белые
        if board.check_winner():
            print("Белые выиграли")
            break

        # получаем ход черных
        while True:
            start = input("Ход черных: ")
            end = input("Ход черных: ")
            if board.is_valid_move(start, end):
                break
            else:
                print("Неправильный ход. Попробуйте снова.")
        
         # ходим черными
        board.move_piece(start, end)
        board.print_board()
        
        # проверяем возможность срубить несколько шашек
        while board.go_again(start, end):
            while True:
                start = input("Ход черных: ")
                end = input("Ход черных: ")
                if board.is_valid_move(start, end):
                    break
                else:
                    print("Неправильный ход. Попробуйте снова.")
            # ходим черными
            board.move_piece(start, end)
            board.print_board()
        
        # проверяем, выиграли ли черные
        if board.check_winner():
            print("Черные выиграли")
            break

play_game()

  a b c d e f g h
8 . c . c . c . c 
7 c . c . c . c . 
6 . c . c . c . c 
5 . . . . . . . . 
4 . . . . . . . . 
3 C . C . C . C . 
2 . C . C . C . C 
1 C . C . C . C . 
Ход белых: e3
Ход белых: f4
  a b c d e f g h
8 . c . c . c . c 
7 c . c . c . c . 
6 . c . c . c . c 
5 . . . . . . . . 
4 . . . . . C . . 
3 C . C . . . C . 
2 . C . C . C . C 
1 C . C . C . C . 
Ход черных: h6
Ход черных: g5
  a b c d e f g h
8 . c . c . c . c 
7 c . c . c . c . 
6 . c . c . c . . 
5 . . . . . . c . 
4 . . . . . C . . 
3 C . C . . . C . 
2 . C . C . C . C 
1 C . C . C . C . 
Ход белых: c3
Ход белых: d4
  a b c d e f g h
8 . c . c . c . c 
7 c . c . c . c . 
6 . c . c . c . . 
5 . . . . . . c . 
4 . . . C . C . . 
3 C . . . . . C . 
2 . C . C . C . C 
1 C . C . C . C . 
Ход черных: g5
Ход черных: e3
  a b c d e f g h
8 . c . c . c . c 
7 c . c . c . c . 
6 . c . c . c . . 
5 . . . . . . . . 
4 . . . C . . . . 
3 C . . . c . C . 
2 . C . C . C . C 
1 C . C . C . C . 
Ход черных: e3
Ход черных: c5
  a 